In [240]:
import sympy as sp

In [247]:
for operation in [ sp.Add,sp.Pow, sp.sin, sp.cos, sp.tan, sp.exp, sp.log]:
    print(operation, operation.nargs)

AttributeError: type object 'Pow' has no attribute 'nargs'

In [248]:
x,y = sp.symbols("x, y")
expr = x + y

In [249]:
expr.func

sympy.core.add.Add